<h1>GPT 5-latest</h1>

<h3>1. Evaluate internal consistency (Cronbach's Alpha) for Advertisement Effectiveness Scores</h3>

In [ ]:
import pingouin as pg
import pandas as pd

def calculate_aes_cronbach_alphas(df):

    alphas = {}

    trait_items = {
        "Product 1 - Openness": ['p_1_openness_item_1', 'p_1_openness_item_2', 'p_1_openness_item_3', 'p_1_openness_item_4', 'p_1_openness_item_5', 'p_1_openness_item_6'],
        "Product 1 - Conscientiousness": ['p_1_consc_item_1', 'p_1_consc_item_2', 'p_1_consc_item_3', 'p_1_consc_item_4', 'p_1_consc_item_5', 'p_1_consc_item_6'],
        "Product 1 - Extraversion": ['p_1_extr_item_1', 'p_1_extr_item_2', 'p_1_extr_item_3', 'p_1_extr_item_4', 'p_1_extr_item_5', 'p_1_extr_item_6'],
        "Product 1 - Agreeableness": ['p_1_agree_item_1', 'p_1_agree_item_2', 'p_1_agree_item_3', 'p_1_agree_item_4', 'p_1_agree_item_5', 'p_1_agree_item_6'],
        "Product 1 - Neuroticism": ['p_1_neuro_item_1', 'p_1_neuro_item_2', 'p_1_neuro_item_3', 'p_1_neuro_item_4', 'p_1_neuro_item_5', 'p_1_neuro_item_6'],
        
        "Product 2 - Openness": ['p_2_openness_item_1', 'p_2_openness_item_2', 'p_2_openness_item_3', 'p_2_openness_item_4', 'p_2_openness_item_5', 'p_2_openness_item_6'],
        "Product 2 - Conscientiousness": ['p_2_consc_item_1', 'p_2_consc_item_2', 'p_2_consc_item_3', 'p_2_consc_item_4', 'p_2_consc_item_5', 'p_2_consc_item_6'],
        "Product 2 - Extraversion": ['p_2_extr_item_1', 'p_2_extr_item_2', 'p_2_extr_item_3', 'p_2_extr_item_4', 'p_2_extr_item_5', 'p_2_extr_item_6'],
        "Product 2 - Agreeableness": ['p_2_agree_item_1', 'p_2_agree_item_2', 'p_2_agree_item_3', 'p_2_agree_item_4', 'p_2_agree_item_5', 'p_2_agree_item_6'],
        "Product 2 - Neuroticism": ['p_2_neuro_item_1', 'p_2_neuro_item_2', 'p_2_neuro_item_3', 'p_2_neuro_item_4', 'p_2_neuro_item_5', 'p_2_neuro_item_6'],
        
        "Product 3 - Openness": ['p_3_openness_item_1', 'p_3_openness_item_2', 'p_3_openness_item_3', 'p_3_openness_item_4', 'p_3_openness_item_5', 'p_3_openness_item_6'],
        "Product 3 - Conscientiousness": ['p_3_consc_item_1', 'p_3_consc_item_2', 'p_3_consc_item_3', 'p_3_consc_item_4', 'p_3_consc_item_5', 'p_3_consc_item_6'],
        "Product 3 - Extraversion": ['p_3_extr_item_1', 'p_3_extr_item_2', 'p_3_extr_item_3', 'p_3_extr_item_4', 'p_3_extr_item_5', 'p_3_extr_item_6'],
        "Product 3 - Agreeableness": ['p_3_agree_item_1', 'p_3_agree_item_2', 'p_3_agree_item_3', 'p_3_agree_item_4', 'p_3_agree_item_5', 'p_3_agree_item_6'],
        "Product 3 - Neuroticism": ['p_3_neuro_item_1', 'p_3_neuro_item_2', 'p_3_neuro_item_3', 'p_3_neuro_item_4', 'p_3_neuro_item_5', 'p_3_neuro_item_6'],
    }

    for trait, item_columns in trait_items.items():
        # Subset the DataFrame to include only the columns of interest
        subset = df[item_columns]
        
        # Check if all required columns are in the DataFrame
        if not all(col in df.columns for col in item_columns):
            print("ALERT!!!")
            alphas[trait] = None  # Assign None if any column is missing
            continue

        # Calculate Cronbach's alpha using pingouin
        alpha = pg.cronbach_alpha(data=subset)
        
        # Store the alpha value in the dictionary
        alphas[trait] = alpha[0]

    return alphas

df = pd.read_csv('../../data/synthetic_participants_gemini_1.5_flash.csv')
alphas = calculate_aes_cronbach_alphas(df)

for product, alpha in alphas.items():
    if alpha is not None:
        print(f"Cronbach's Alpha for {product}: {alpha:.3f}")
    else:
        print(f"Cronbach's Alpha for {product}: Data missing")


res = df.isna().any(axis=1).sum()
res

<h3>2. Aggregate Advertisement Effectiveness Scores (AES) scores to derive Dependent Variables: AES by trait and product</h3>

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

def calculate_raw_and_residualized_aes(dataframe):
    product_numbers = [1, 2, 3]
    
    traits = {
        "openness": "openness",
        "conscientiousness": "consc",
        "extraversion": "extr",
        "agreeableness": "agree",
        "neuroticism": "neuro"
    }

    for product_num in product_numbers:
        # Dictionary to store raw AES columns for each trait for the current product
        aes_columns = {}

        # Step 1: Calculate Raw AES
        for trait_name, trait_prefix in traits.items():
            # Find all relevant item columns for this product and trait
            target_columns = [
                col for col in dataframe.columns
                if col.startswith(f"p_{product_num}_{trait_prefix}_item_")
            ]
            
            if target_columns:
                # Calculate raw AES as the mean of relevant columns
                dataframe[f"aes_{product_num}_{trait_name}"] = dataframe[target_columns].mean(axis=1, skipna=True)
                aes_columns[trait_name] = dataframe[f"aes_{product_num}_{trait_name}"]
            else:
                print(f"No valid columns found for product {product_num}, trait {trait_name}!")

        # Step 2: Calculate Residualized AES
        for target_trait, aes_target in aes_columns.items():
            # Use raw AES scores of other traits as predictors
            predictors = [
                aes_columns[other_trait]
                for other_trait in traits.keys()
                if other_trait != target_trait and other_trait in aes_columns
            ]

            if predictors:
                # Stack predictors into a matrix
                predictors_matrix = np.column_stack(predictors)
                
                # Perform regression to calculate residuals
                regression_model = LinearRegression()
                regression_model.fit(predictors_matrix, aes_target)
                residuals = aes_target - regression_model.predict(predictors_matrix)
                
                # Save residualized AES
                dataframe[f"aes_resd_{product_num}_{target_trait}"] = residuals
            else:
                # If no predictors are available, retain raw AES
                dataframe[f"aes_{product_num}_{target_trait}"] = aes_target
                print(f"Could not residualize AES for product {product_num}, trait {target_trait} due to missing predictors.")

    return dataframe

df = calculate_raw_and_residualized_aes(df.copy())


<h1>Analysis</h1>
<h3>1. Regression Analysis: using Big Five personality traits to predict respondents’ AES scores</h3>

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

def perform_regression_on_personality(df, aes_type='raw'):
    # Personality trait columns (predictors)
    traits_cols = [
        'extraversion_score',
        'agreeableness_score',
        'conscientiousness_score',
        'neuroticism_score',
        'openness_score'
    ]

    if aes_type == 'raw':
        aes_sufix = 'aes'
    elif aes_type == 'resd':
        aes_sufix = 'aes_resd'

    # AES columns grouped by product
    product_aes_cols = {
        "p1": [f'{aes_sufix}_1_extraversion', f'{aes_sufix}_1_agreeableness', f'{aes_sufix}_1_conscientiousness', f'{aes_sufix}_1_neuroticism', f'{aes_sufix}_1_openness'],
        "p2": [f'{aes_sufix}_2_extraversion', f'{aes_sufix}_2_agreeableness', f'{aes_sufix}_2_conscientiousness', f'{aes_sufix}_2_neuroticism', f'{aes_sufix}_2_openness'],
        "p3": [f'{aes_sufix}_3_extraversion', f'{aes_sufix}_3_agreeableness', f'{aes_sufix}_3_conscientiousness', f'{aes_sufix}_3_neuroticism', f'{aes_sufix}_3_openness']
    }

    results_dict = {}

    for product, aes_cols in product_aes_cols.items():
        # Create a results DataFrame for this product
        results = pd.DataFrame(index=traits_cols, columns=aes_cols)

        for aes_col in aes_cols:
            # Standardize predictors (traits) and outcome (AES)
            scaler = StandardScaler()
            X = scaler.fit_transform(df[traits_cols])
            y = scaler.fit_transform(df[[aes_col]]).flatten()

            # Add constant to predictors
            X = sm.add_constant(X)

            # Fit regression model
            model = sm.OLS(y, X).fit()

            # Extract coefficients and p-values (skip constant)
            coefficients = model.params[1:]
            p_values = model.pvalues[1:]

            # Store results as standardized beta coefficients with p-values
            results[aes_col] = [
                f"{0.00 if round(coeff, 2) == 0 else coeff:.2f} ({pval:.4f})"
                for coeff, pval in zip(coefficients, p_values)
            ]

        # Save this product's results
        results_dict[product] = results

    # Return results for all three products
    return results_dict["p1"], results_dict["p2"], results_dict["p3"]


# Raw AES
raw_results_p1, raw_results_p2, raw_results_p3 = perform_regression_on_personality(df, aes_type='raw')

print("GPT-5-latest: Regression Coefficient Matrix – P1: Cabin luggage")
display(raw_results_p1)
print("GPT-5-latest: Regression Coefficient – P2: Packing Cubes")
display(raw_results_p2)
print("GPT-5-latest: Regression Coefficient – P3:Water Bottle")
display(raw_results_p3)

# # Residualized AES
resd_results_p1, resd_results_p2, resd_results_p3 = perform_regression_on_personality(df, aes_type='resd')

print("GPT-5-latest: Regression Coefficient Matrix – P1: Cabin luggage (Residualized AES)")
display(resd_results_p1)
print("GPT-5-latest: Regression Coefficient – P2: Packing Cubes (Residualized AES)")
display(resd_results_p2)
print("GPT-5-latest: Regression Coefficient – P3:Water Bottle (Residualized AES)")
display(resd_results_p3)


<h3>2. Combine human and syntheti twin agents datasets</h3>

In [ ]:
import pandas as pd

df_humans = pd.read_csv('../../data/filtered_participants_dataset.csv')
df_synths = df.copy()

# Combine both DataFrames
df_combined = pd.concat([df_humans, df_synths], axis=0).reset_index(drop=True)

df_humans['Group'] = 'Humans'
df_synths['Group'] = 'Synthetic Twins'

# Define AES and personality trait columns
aes_cols_product1 = ['aes_1_extraversion', 'aes_1_agreeableness', 'aes_1_conscientiousness', 'aes_1_neuroticism', 'aes_1_openness']
aes_cols_product2 = ['aes_2_extraversion', 'aes_2_agreeableness', 'aes_2_conscientiousness', 'aes_2_neuroticism', 'aes_2_openness']
aes_cols_product3 = ['aes_3_extraversion', 'aes_3_agreeableness', 'aes_3_conscientiousness', 'aes_3_neuroticism', 'aes_3_openness']

personality_cols = ['extraversion_score', 'agreeableness_score', 'conscientiousness_score', 'neuroticism_score', 'openness_score']

df_combined = pd.concat([df_humans, df_synths], axis=0).reset_index(drop=True)

<h3>3. Effects (with 95% confidence intervals) of Big Five personality traits on effectiveness ratings of all three product ads for human participants and synthetic twins </h3>

In [ ]:
import numpy as np
from scipy.stats import linregress, t
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_scatter_plots_with_fit_grouped(df_combined, aes_cols, traits_cols, product_name,
                                          confidence=0.95, alpha=0.020397):
    """alpha is the decision cutoff (our FDR-adjusted threshold)."""
    traits_order = ['openness','conscientiousness','extraversion','agreeableness','neuroticism']
    fig = make_subplots(
        rows=len(traits_order), cols=2,
        subplot_titles=["Humans" if i % 2 == 0 else "Synthetic Twins"
                        for trait in traits_order for i in range(2)],
        vertical_spacing=0.12, horizontal_spacing=0.12
    )

    def format_p(p, threshold=alpha):
        return f"< {threshold:.6f}".replace("0.", ".") if p < threshold else f"= {p:.6f}".replace("0.", ".")

    for i, trait in enumerate(traits_order):
        trait_col = f"{trait}_score"
        aes_col = next((c for c in aes_cols if c.endswith(trait)), None)
        if aes_col is None or trait_col not in traits_cols:
            continue

        for j, group in enumerate(["Humans", "Synthetic Twins"]):
            data = df_combined[df_combined["Group"] == group][[trait_col, aes_col]].dropna()
            if data.empty:
                continue

            x = data[trait_col].values
            y = data[aes_col].values
            x_reshaped = x.reshape(-1, 1)

            # Fit model & CI for the line
            lin_model = LinearRegression().fit(x_reshaped, y)
            y_pred = lin_model.predict(x_reshaped)
            slope, intercept, r_value, p_value, std_err = linregress(x, y)

            n = len(x)
            mean_x = np.mean(x)
            t_val = t.ppf((1 + confidence) / 2., df=n - 2)
            residuals = y - y_pred
            se = np.sqrt(np.sum(residuals ** 2) / (n - 2))
            se_line = se * np.sqrt(1/n + ((x - mean_x)**2 / np.sum((x - mean_x)**2)))
            ci_upper = y_pred + t_val * se_line
            ci_lower = y_pred - t_val * se_line

            # Sort for plotting
            sorted_idx = np.argsort(x)
            x_sorted = x[sorted_idx]
            y_sorted = y_pred[sorted_idx]
            ci_upper_sorted = ci_upper[sorted_idx]
            ci_lower_sorted = ci_lower[sorted_idx]

            row, col = i + 1, j + 1

            # Scatter
            fig.add_trace(go.Scatter(
                x=x, y=y, mode='markers',
                marker=dict(color='black', opacity=0.3),
                showlegend=False
            ), row=row, col=col)

            # Regression line
            fig.add_trace(go.Scatter(
                x=x_sorted, y=y_sorted, mode='lines',
                line=dict(color='blue'),
                showlegend=False
            ), row=row, col=col)

            # Confidence band
            fig.add_trace(go.Scatter(
                x=np.concatenate([x_sorted, x_sorted[::-1]]),
                y=np.concatenate([ci_upper_sorted, ci_lower_sorted[::-1]]),
                fill='toself', fillcolor='rgba(0,0,0,0.15)',
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo="skip", showlegend=False
            ), row=row, col=col)

            # Axes
            fig.update_xaxes(title_text="Advertisement Effectiveness Score", row=row, col=col)
            fig.update_yaxes(title_text=f"{trait.capitalize()}-Tailored Ad", row=row, col=col)

            # def format_p(p, decimals=4):
            #     return f"= {p:.{decimals}f}"

            # Annotation: bold Significant / Non-Significant + p-value
            sig = p_value < alpha
            label = "<b>Significant</b>" if sig else "<b>Non-Significant</b>"
            color = "green" if sig else "red"
            text = f"{label} (p {format_p(p_value)})"
            
            fig.add_annotation(
                row=row, col=col, xref="x domain", yref="y domain",
                x=0.02, y=0.98, showarrow=False, align="left",
                text=text, font=dict(size=14, color=color),
                bgcolor="rgba(255,255,255,0.9)"
            )

    fig.update_layout(
        height=300 * len(traits_order),
        width=1400,
        title_text=f"{product_name} – AES Scores: Humans vs Synthetic Twins (GPT-5-latest)",
        template="simple_white",
        showlegend=False
    )
    return fig


In [ ]:
personality_cols = ['extraversion_score', 'agreeableness_score', 'conscientiousness_score', 'neuroticism_score', 'openness_score']

aes_cols_product1 = ['aes_1_extraversion', 'aes_1_agreeableness', 'aes_1_conscientiousness', 'aes_1_neuroticism', 'aes_1_openness']
aes_cols_product2 = ['aes_2_extraversion', 'aes_2_agreeableness', 'aes_2_conscientiousness', 'aes_2_neuroticism', 'aes_2_openness']
aes_cols_product3 = ['aes_3_extraversion', 'aes_3_agreeableness', 'aes_3_conscientiousness', 'aes_3_neuroticism', 'aes_3_openness']

fig1 = create_scatter_plots_with_fit_grouped(df_combined, aes_cols_product1, personality_cols, "Cabin Luggage")
fig1.show()

fig2 = create_scatter_plots_with_fit_grouped(df_combined, aes_cols_product2, personality_cols, "Packing Cubes")
fig2.show()

fig3 = create_scatter_plots_with_fit_grouped(df_combined, aes_cols_product3, personality_cols, "Water Bottle")
fig3.show()

<h3>4. Compute Confidence Intervals</h3>

In [ ]:
import pandas as pd
import numpy as np
import pingouin as pg
from scipy.stats import pearsonr

# Define products and traits
products = {
    "Cabin Luggage": "aes_1_",
    "Packing Cubes": "aes_2_",
    "Water Bottle": "aes_3_"
}

traits = ['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']

def boot_ci(data):
    data = data.dropna()
    if len(data) < 2:
        return None, None
    ci = pg.compute_bootci(data, func='mean', n_boot=1000, confidence=0.95)
    return data.mean(), (ci[0], ci[1])

# Output rows + p-values collectors
table_rows = []
pvals_trait = []   # raw p-values for each trait t-test
pvals_agg = []     # raw p-values for aggregate Pearson correlation


for product_name, prefix in products.items():
    human_means = []
    synth_means = []
    same_cat_count = 0

    for trait in traits:
        col_name = f"{prefix}{trait}"
        trait_label = trait.capitalize()

        # Subset data
        humans = df_combined[df_combined["Group"] == "Humans"][col_name]
        synths = df_combined[df_combined["Group"] == "Synthetic Twins"][col_name]

        # Compute CIs and means
        h_mean, h_ci = boot_ci(humans)
        s_mean, s_ci = boot_ci(synths)

        # Store means for correlation later
        human_means.append(h_mean)
        synth_means.append(s_mean)

        # Categories
        h_cat = int(round(h_mean)) if h_mean is not None else None
        s_cat = int(round(s_mean)) if s_mean is not None else None
        same_cat = "✅" if h_cat == s_cat else "❌"
        if same_cat == "✅":
            same_cat_count += 1

        # T-test
        if humans.dropna().shape[0] > 1 and synths.dropna().shape[0] > 1:
            pval = pg.ttest(humans.dropna(), synths.dropna(), paired=False)['p-val'].values[0]
            pvals_trait.append(pval)
            pval_str = f"{pval:.3f}"
        else:
            pvals_trait.append(np.nan)
            pval_str = "N/A"

        # Format means and CIs
        h_str = f"{h_mean:.2f} [{h_ci[0]:.2f}, {h_ci[1]:.2f}]" if h_mean is not None else "N/A"
        s_str = f"{s_mean:.2f} [{s_ci[0]:.2f}, {s_ci[1]:.2f}]" if s_mean is not None else "N/A"

        # Append row
        table_rows.append({
            "Product": product_name,
            "Trait": trait_label,
            "Human Mean [95% CI]": h_str,
            "Synthetic Mean [95% CI]": s_str,
            "Human Cat": h_cat,
            "Synthetic Cat": s_cat,
            "p-value": pval_str,
            "Same Category": same_cat,
            "Aggregate r": "", "Agg. p": "", "% Same Category": ""
        })

    # Aggregate correlation per product
    if None not in human_means and None not in synth_means:
        r, p = pearsonr(human_means, synth_means)
        r = round(r, 2)
        p_agg = round(p, 3)
        pvals_agg.append(p)  # raw p-value

        same_percent = round((same_cat_count / len(traits)) * 100, 1)
        for i in range(len(traits)):
            table_rows[-(i+1)]["Aggregate r"] = r
            table_rows[-(i+1)]["Agg. p"] = p_agg
            table_rows[-(i+1)]["% Same Category"] = same_percent
    else:
        pvals_agg.append(np.nan)

# Create DataFrame
final_df = pd.DataFrame(table_rows)

# Display results
from IPython.display import display
display(final_df)


<h3>5. Store p-values for for p-val FDR correction</h3>

In [ ]:
import json

pvals = {
    "gpt-5-latest-flash-ci-test": pvals_trait,
}

# save pvals
with open("../p_value_correction/gpt_5_latest_ci.json", "w") as f:
    json.dump(pvals, f, indent=4)

<h3>6. Compute Pearson Correaltions for Fisher's z test </h3>

In [ ]:
import pandas as pd
import pingouin as pg

def compute_correlations_with_pvalues(df, aes_type='raw'):
    # Big Five traits
    traits_cols = [
        'extraversion_score',
        'agreeableness_score',
        'conscientiousness_score',
        'neuroticism_score',
        'openness_score'
    ]

    # AES suffix
    if aes_type == 'raw':
        aes_suffix = 'aes'
    elif aes_type == 'resd':
        aes_suffix = 'aes_resd'
    else:
        raise ValueError("aes_type must be 'raw' or 'resd'.")

    # AES columns grouped by product
    product_aes_cols = {
        "p1": [f'{aes_suffix}_1_extraversion', f'{aes_suffix}_1_agreeableness',
               f'{aes_suffix}_1_conscientiousness', f'{aes_suffix}_1_neuroticism', f'{aes_suffix}_1_openness'],
        "p2": [f'{aes_suffix}_2_extraversion', f'{aes_suffix}_2_agreeableness',
               f'{aes_suffix}_2_conscientiousness', f'{aes_suffix}_2_neuroticism', f'{aes_suffix}_2_openness'],
        "p3": [f'{aes_suffix}_3_extraversion', f'{aes_suffix}_3_agreeableness',
               f'{aes_suffix}_3_conscientiousness', f'{aes_suffix}_3_neuroticism', f'{aes_suffix}_3_openness']
    }

    results = {}
    p_vals = {}

    for product, aes_cols in product_aes_cols.items():
        correlation_results = pd.DataFrame(index=traits_cols, columns=aes_cols)

        p_vals_product = []

        for trait_col in traits_cols:
            for aes_col in aes_cols:
                corr_res = pg.corr(df[trait_col], df[aes_col])
                correlation = corr_res["r"].values[0]
                p_value = corr_res["p-val"].values[0]
                correlation_results.loc[trait_col, aes_col] = f"{correlation:.2f} (p={p_value:.6f})"
                p_vals_product.append(p_value)

        results[product] = correlation_results
        p_vals[product] = p_vals_product

    return results, p_vals

results, p_vals = compute_correlations_with_pvalues(df, aes_type='raw')


# Display results
print("Pearson Correlations with p-values - Product 1")
display(results["p1"])
print("Pearson Correlations with p-values - Product 2")
display(results["p2"])
print("Pearson Correlations with p-values - Product 3")
display(results["p3"])